In [1]:
import requests
import json

In [5]:
url = 'https://www150.statcan.gc.ca/t1/wds/rest/getFullTableDownloadCSV/14100287/en'

params = {
}

api_result = requests.get(url=url, params=params)

api_response = api_result.json()

print(api_response)

{'status': 'SUCCESS', 'object': 'https://www150.statcan.gc.ca/n1/tbl/csv/14100287-eng.zip'}


# Retrieve time series data for montht

In [3]:
# retrieve data for monthly average retail prices for selected products (table id # 1810024501)
url1 = 'https://www150.statcan.gc.ca/t1/wds/rest/getFullTableDownloadCSV/18100245/en'

params = {
}

api_result = requests.get(url=url1, params=params)

api_response = api_result.json()

print(api_response)

{'status': 'SUCCESS', 'object': 'https://www150.statcan.gc.ca/n1/tbl/csv/18100245-eng.zip'}


In [4]:
# retrieve data for farm product prices, crops and livestock (table id # 3210007701)
url2 = 'https://www150.statcan.gc.ca/t1/wds/rest/getFullTableDownloadCSV/32100077/en'

params = {
}

api_result = requests.get(url=url2, params=params)

api_response = api_result.json()

print(api_response)

{'status': 'SUCCESS', 'object': 'https://www150.statcan.gc.ca/n1/tbl/csv/32100077-eng.zip'}


In [ ]:
# retrieve data for meat, fruits/veg
# no API resource for AgriCanada

In [ ]:
# POST URL = 'https://www150.statcan.gc.ca/t1/wds/rest/getDataFromVectorsAndLatestNPeriods'

# POST BODY = [{"vectorId":1353834271, "latestN":3}]

# Both methods return the same results.

Ideas about what functions/features the tool will have:
- predict cost fluctuations in food products
    - Clarify: what LOD? Category or individual products? (individual products would not be a comprehensive resource; not all ingredients are available with StatsCan database)
    - If prediction at the categorical level:
        - users would input their menu items, including ingredients
            - if the user has an ingredient not listed in our database, they can simply select its category
        - we will use Stats Canada retail database to predict monthly price average for its selected products
        
        - our tool would break down their menu into weighted average by category (30% wheat products, 25% protein, etc)

- two stages of analysis:
    - time-series prediction:
        - use historic product price data to predict fluctuations in monthly average cost
    - farm product price data:
        - track changes in supplier sale prices
        - if suppliers start selling at higher/lower price points, that percentage change will also be applied to our tool's predicted product price
